In [1]:

import plotly.express as px
import pandas as pd
import json
import os
os.environ['USER'] = "rmoine"
from pathlib import Path
import numpy as np
import re
from typing import *
from llama.main import compute_metrics_from_files, DatasetName, compute_metrics_from_list, get_tokenizer
import llama.main as m
from sklearn.metrics import f1_score, auc, roc_curve
from PIL import Image
from pandasql import sqldf
from io import BytesIO
import base64
from plotly.graph_objects import Figure
import win32clipboard
import textwrap
import colorsys
import optuna
import tqdm
def img_to_clipboard(fig: Figure):
    img_bytes = fig.to_image(format="png")
    image = Image.open(BytesIO(img_bytes))
    
    output = BytesIO()
    image.convert('RGB').save(output, 'BMP')
    data = output.getvalue()[14:]
    output.close()
    win32clipboard.OpenClipboard()
    win32clipboard.EmptyClipboard()
    win32clipboard.SetClipboardData(win32clipboard.CF_DIB, data)
    win32clipboard.CloseClipboard()
folder_name = "inference"
root = Path(f"../../data/{folder_name}/")

c:\Users\robin\miniconda3\envs\severityPrediction\lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'
Import of import evaluate failed
Import of from pytorchtools import EarlyStopping failed


In [2]:
split = {}
for dataset_choice in ["eclipse_72k","mozilla_200k"]:
    with open(f"../../data/split_{dataset_choice}.json") as fp:
        split[dataset_choice] = {}
        for dataset_type, bug_ids in json.load(fp).items():
            for bug_id in bug_ids:
                split[dataset_choice][bug_id] = dataset_type

In [3]:

dico_inferences_prompt = {}
tokenizer = get_tokenizer(m.default_token, m.default_model)
pathes = list(root.rglob("inference__*"))
for path_data in tqdm.tqdm(pathes):
    with open(path_data / "parameters.json") as fp:
        parameters = json.load(fp)
    dataset_choice = parameters["dataset_choice"]
    files_data = list(path_data.rglob(f"predictions_*.json"))
    if len(files_data) == 0:
        continue
    for f in files_data:
        d = []
        with open(f) as fp:
            for line in fp.readlines():
                if line.strip() == '':
                    continue
                try:
                    line = json.loads(line)
                except Exception as e:
                    print(e)
                    print(line)
                    raise Exception
                if parameters['prompt_id'] not in dico_inferences_prompt:
                    dico_inferences_prompt[parameters['prompt_id']] = []
                dico_inferences_prompt[parameters['prompt_id']].append({**line,"dataset_type": split[dataset_choice][line['bug_id']],"dataset_choice":dataset_choice,"parameters":{**parameters,"path_data_folder": path_data.stem}})
dfs = {k:pd.DataFrame(v) for k,v in dico_inferences_prompt.items()}
    


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\robin\.cache\huggingface\token
Login successful


100%|██████████| 300/300 [03:59<00:00,  1.25it/s]


In [4]:
for k,df_elems in dfs.items():
    df_elems.drop_duplicates(subset=["dataset_choice","bug_id"], keep='last',inplace=True)
    df_elems = df_elems.dropna(subset=["binary_severity","severity_pred"])
    l = []
    df_elems["n_tokens_infered_max"] = df_elems["parameters"].apply(lambda x:x["n_tokens_infered_max"])
    for dataset_choice in df_elems["dataset_choice"].unique():
        for n_tokens_infered_max in df_elems["n_tokens_infered_max"].unique():
            for dataset_type in df_elems["dataset_type"].unique():
                d = df_elems.query(f"dataset_choice == '{dataset_choice}' & n_tokens_infered_max == {n_tokens_infered_max} & dataset_type == '{dataset_type}'").to_dict(orient="records")
                confusion_matrix, f1, _ = compute_metrics_from_list(d, pred_field="severity_pred",)
                accuracy = np.sum(np.diag(confusion_matrix)) / np.sum(confusion_matrix)
                df1 = pd.DataFrame(d)
                try:
                    fpr, tpr, thresholds = roc_curve(y_true=np.array(df1['binary_severity']), y_score=np.array(df1['severity_pred']))
                    roc_auc = auc(fpr, tpr)
                    l.append({"n_elems": len(d), "n_tokens_infered_max": n_tokens_infered_max, "roc_auc":roc_auc, "dataset_choice": dataset_choice, "dataset_type":dataset_type,"confusion_matrix":confusion_matrix,"f1":f1,"accuracy":accuracy*100, **d[0]["parameters"]})
                except ValueError:
                    display(df1[df1.isna()])
                    raise Exception
            df1 = df_elems.query(f"dataset_choice == '{dataset_choice}' & n_tokens_infered_max == {n_tokens_infered_max}")
            with open(f"../../data/{folder_name}/inference_{dataset_choice}_prompt-{k}_missing.json", "w") as fp:
                json.dump([int(i) for i in sorted(df1['bug_id'].unique())], fp)
    df = pd.DataFrame(l)
    df_elems.to_json(f"../../data/{folder_name}/predictions_prompt-{k}.json")
    df.to_json(f"../../data/{folder_name}/metrics_prompt-{k}.json")
    print(k)
    display(df.head())

alpaca


,n_elems,n_tokens_infered_max,roc_auc,dataset_choice,dataset_type,confusion_matrix,f1,accuracy,folder_data,model_name,...,seed_end,n_chunks,interval_idx,id_name,field_input,missing_file,prompt_id,folder_out,path_data_json,path_data_folder
0,50534,1000,0.472522,eclipse_72k,tr,"[[7282, 5200, 182], [23961, 13194, 715]]","[0.0, 0.3348573636422178, 0.03688704310367065]",40.519254,/project/6023391/rmoine/data,meta-llama/Llama-2-13b-chat-hf,...,None,50,0,_23709732,description,,alpaca,/project/6023391/rmoine/data/inference__23709732,/project/6023391/rmoine/data/eclipse_72k.json,inference__23709732
1,7220,1000,0.478487,eclipse_72k,test,"[[1046, 738, 26], [3382, 1905, 123]]","[0.0, 0.33146193577363575, 0.044252563410685376]",40.872576,/project/6023391/rmoine/data,meta-llama/Llama-2-13b-chat-hf,...,None,50,0,_23709732,description,,alpaca,/project/6023391/rmoine/data/inference__23709732,/project/6023391/rmoine/data/eclipse_72k.json,inference__23709732
2,14438,1000,0.465880,eclipse_72k,val,"[[2044, 1522, 52], [6882, 3737, 201]]","[0.0, 0.34290864030640983, 0.03630452451910052]",40.040172,/project/6023391/rmoine/data,meta-llama/Llama-2-13b-chat-hf,...,None,50,0,_23709732,description,,alpaca,/project/6023391/rmoine/data/inference__23709732,/project/6023391/rmoine/data/eclipse_72k.json,inference__23709732
3,128438,1000,0.494190,mozilla_200k,tr,"[[20159, 12244, 758], [59412, 32793, 3072]]","[0.0, 0.3131537890994655, 0.061993602873661804]",41.227674,/project/6023391/rmoine/data,meta-llama/Llama-2-13b-chat-hf,...,None,50,0,_23709901,description,,alpaca,/project/6023391/rmoine/data/inference__23709901,/project/6023391/rmoine/data/mozilla_200k.json,inference__23709901
4,18425,1000,0.486928,mozilla_200k,test,"[[2852, 1808, 118], [8551, 4664, 432]]","[0.0, 0.32142222222222216, 0.06085792773121082]",40.792402,/project/6023391/rmoine/data,meta-llama/Llama-2-13b-chat-hf,...,None,50,0,_23709901,description,,alpaca,/project/6023391/rmoine/data/inference__23709901,/project/6023391/rmoine/data/mozilla_200k.json,inference__23709901


official


,n_elems,n_tokens_infered_max,roc_auc,dataset_choice,dataset_type,confusion_matrix,f1,accuracy,folder_data,model_name,...,seed_end,n_chunks,interval_idx,id_name,field_input,missing_file,prompt_id,folder_out,path_data_json,path_data_folder
0,50534,1000,0.544388,eclipse_72k,tr,"[[0, 1655, 11009], [1, 1586, 36283]]","[0.0, 0.2081106570260924, 0.8520936567952843]",3.138481,/project/6023391/rmoine/data,meta-llama/Llama-2-13b-chat-hf,...,None,50,0,_23709823,description,,official,/project/6023391/rmoine/data/inference__23709823,/project/6023391/rmoine/data/eclipse_72k.json,inference__23709823
1,7220,1000,0.548632,eclipse_72k,test,"[[254, 1556], [233, 5177]]","[0.22115803221593378, 0.8526723215020999]",75.221607,/project/6023391/rmoine/data,meta-llama/Llama-2-13b-chat-hf,...,None,50,0,_23709823,description,,official,/project/6023391/rmoine/data/inference__23709823,/project/6023391/rmoine/data/eclipse_72k.json,inference__23709823
2,14438,1000,0.540101,eclipse_72k,val,"[[449, 3169], [475, 10345]]","[0.19771025979744605, 0.8502506780636148]",74.761047,/project/6023391/rmoine/data,meta-llama/Llama-2-13b-chat-hf,...,None,50,0,_23709823,description,,official,/project/6023391/rmoine/data/inference__23709823,/project/6023391/rmoine/data/eclipse_72k.json,inference__23709823
3,139651,1000,0.529177,mozilla_200k,tr,"[[0, 3507, 32926], [5, 3907, 99306]]","[0.0, 0.15996533400232626, 0.8435421533234231]",2.797689,/project/6023391/rmoine/data,meta-llama/Llama-2-13b-chat-hf,...,None,50,0,_23710031,description,,official,/project/6023391/rmoine/data/inference__23710031,/project/6023391/rmoine/data/mozilla_200k.json,inference__23710031
4,19951,1000,0.530729,mozilla_200k,test,"[[0, 519, 4686], [1, 563, 14182]]","[0.0, 0.1651025926515031, 0.8438150770512287]",2.821914,/project/6023391/rmoine/data,meta-llama/Llama-2-13b-chat-hf,...,None,50,0,_23710031,description,,official,/project/6023391/rmoine/data/inference__23710031,/project/6023391/rmoine/data/mozilla_200k.json,inference__23710031


In [5]:
df.iloc[0].confusion_matrix

array([[    0,  1655, 11009],
       [    1,  1586, 36283]], dtype=int64)

In [6]:
dfs = {}
for p in Path(f"../../data/{folder_name}/").rglob("*_prompt-*.json"):
    if not p.is_file():
        continue
    prompt_types = re.findall("^.*prompt-(.*)",p.stem)
    if len(prompt_types) != 1 or prompt_types[0] not in ['alpaca','official']:
        continue
    [prompt_type] = prompt_types
    if prompt_type not in dfs:
        dfs[prompt_type] = {"df": None, "df_elems": None}
    type_df = re.findall("^([^_]+)_", p.stem)
    if len(type_df) != 1:
        continue
    [type_df] = type_df
    print(p, prompt_type, type_df)
    if type_df == "metrics":
        dfs[prompt_type]['df'] = pd.read_json(p)
    elif type_df == "predictions":
        dfs[prompt_type]['df_elems'] = pd.read_json(p)

..\..\data\inference\metrics_prompt-alpaca.json alpaca metrics
..\..\data\inference\metrics_prompt-official.json official metrics
..\..\data\inference\predictions_prompt-alpaca.json alpaca predictions
..\..\data\inference\predictions_prompt-official.json official predictions


In [7]:

for prompt_type in dfs:
    for i in range(len(dfs[prompt_type]['df'])):
        dataset_type = dfs[prompt_type]['df'].iloc[i].dataset_type
        dataset_choice = dfs[prompt_type]['df'].iloc[i].dataset_choice
        n_tokens_infered_max = dfs[prompt_type]['df'].iloc[i].n_tokens_infered_max
        data = dfs[prompt_type]['df'].iloc[i]
        data_full = dfs[prompt_type]['df_elems'].query(f"dataset_choice == '{dataset_choice}' & dataset_type == '{dataset_type}' & n_tokens_infered_max == {n_tokens_infered_max}")
        # display(data_full.query("binary_severity == 1"))
        all_bug_ids = set(pd.read_json(f"../../data/llm/{dataset_choice}.json")['bug_id'])
        missing = (all_bug_ids.difference(set(data_full['bug_id'])))
        # add the nan predictions
        d = data_full.query("severity_pred < -1")
        missing = list(missing.union(d['bug_id']))
        with open(f'../../data/{folder_name}/inference_{dataset_type}_prompt-{prompt_type}_missing.json', 'w') as fp:
            json.dump(missing,fp,indent=4)
        possibilities_pred = list(sorted(set(data_full["severity_pred"].unique())))
        print(data.confusion_matrix)
        compute_metrics_from_files(np.array(data.confusion_matrix),
                                f1=data.f1,
                                folder_out=Path(f"../../data/{folder_name}/"),
                                possibilities_pred=possibilities_pred,
                                id="_"+dataset_choice+"_"+dataset_type+"_"+str(n_tokens_infered_max)+f"_prompt-{prompt_type}",
                                title=f"Confusion matrix inference\n{dataset_type}\nprompt limit to {n_tokens_infered_max} tokens",
                                n_tokens_infered_max=n_tokens_infered_max,
                                data_full=data_full,
                                )


[[7282, 5200, 182], [23961, 13194, 715]]


c:\Users\robin\Documents\projets\severityPrediction\src\llm\llama\main.py:912: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


[[1046, 738, 26], [3382, 1905, 123]]


c:\Users\robin\Documents\projets\severityPrediction\src\llm\llama\main.py:912: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


[[2044, 1522, 52], [6882, 3737, 201]]


...

c:\Users\robin\Documents\projets\severityPrediction\src\llm\llama\main.py:912: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


[[0, 519, 4686], [1, 563, 14182]]


c:\Users\robin\Documents\projets\severityPrediction\src\llm\llama\main.py:912: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


[[1, 1016, 9393], [1, 1099, 28391]]


c:\Users\robin\Documents\projets\severityPrediction\src\llm\llama\main.py:912: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [ ]:
df_oom = df_elems.query(f"dataset_type == 'mozilla_200k' & n_tokens_infered_max == 1000 & severity_pred == -2")
pd.set_option('display.max_colwidth', None)
display(df_oom.query("n_tokens <= 148")[['n_tokens','input']].sort_values("n_tokens"))
px.histogram(df_oom, x="n_tokens", title="n_tokens (template included) for mozilla_200k samples that yield oom")

,n_tokens,input
221189,146,Always answer with one token. Do not give any explanation. Use only 0 or 1 and one token. Skip any politeness answer. You have only one word available.\nBelow is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nCategorize the bug report into one of 2 categories:\n\n0 = NOT SEVERE\n1 = SEVERE\n\n### Input: proxy\n\n\n### Remembering the instruction:\nCategorize the bug report into one of 2 categories:\n\n0 = NOT SEVERE\n1 = SEVERE\n\n### Response:
224834,146,Always answer with one token. Do not give any explanation. Use only 0 or 1 and one token. Skip any politeness answer. You have only one word available.\nBelow is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nCategorize the bug report into one of 2 categories:\n\n0 = NOT SEVERE\n1 = SEVERE\n\n### Input: run\n\n\n### Remembering the instruction:\nCategorize the bug report into one of 2 categories:\n\n0 = NOT SEVERE\n1 = SEVERE\n\n### Response:
233878,146,Always answer with one token. Do not give any explanation. Use only 0 or 1 and one token. Skip any politeness answer. You have only one word available.\nBelow is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nCategorize the bug report into one of 2 categories:\n\n0 = NOT SEVERE\n1 = SEVERE\n\n### Input: crashes\n\n\n### Remembering the instruction:\nCategorize the bug report into one of 2 categories:\n\n0 = NOT SEVERE\n1 = SEVERE\n\n### Response:
216009,146,Always answer with one token. Do not give any explanation. Use only 0 or 1 and one token. Skip any politeness answer. You have only one word available.\nBelow is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nCategorize the bug report into one of 2 categories:\n\n0 = NOT SEVERE\n1 = SEVERE\n\n### Input: Browser\n\n\n### Remembering the instruction:\nCategorize the bug report into one of 2 categories:\n\n0 = NOT SEVERE\n1 = SEVERE\n\n### Response:
226258,146,Always answer with one token. Do not give any explanation. Use only 0 or 1 and one token. Skip any politeness answer. You have only one word available.\nBelow is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nCategorize the bug report into one of 2 categories:\n\n0 = NOT SEVERE\n1 = SEVERE\n\n### Input: windows\n\n\n### Remembering the instruction:\nCategorize the bug report into one of 2 categories:\n\n0 = NOT SEVERE\n1 = SEVERE\n\n### Response:
...,...,...
221378,148,Always answer with one token. Do not give any explanation. Use only 0 or 1 and one token. Skip any politeness answer. You have only one word available.\nBelow is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nCategorize the bug report into one of 2 categories:\n\n0 = NOT SEVERE\n1 = SEVERE\n\n### Input: Suggestion\n\n\n### Remembering the instruction:\nCategorize the bug report into one of 2 categories:\n\n0 = NOT SEVERE\n1 = SEVERE\n\n### Response:
221334,148,Always answer with one token. Do not give any explanation. Use only 0 or 1 and one token. Skip any politeness answer. You have only one word available.\nBelow is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nCategorize the bug report into one of 2 categories:\n\n0 = NOT SEVERE\n1 = SEVERE\n\n### Input: news mail hang\n\n\n### Remembering the instruction:\nCategorize the bug report into one of 2 categories:\n\n0 = NOT SEVERE\n1 = SEVERE\n\n### Response:
221317,148,Always answer with one token. Do not give any explanation. Use only 0 or 1 and one token. Skip any politeness answer. You have only one word available.\nBelow is an instruction that describes a task. Write a response that appr